In [ ]:
# Install dependencies
#!pip install python-magic python-docx pdfplumber requests

import os
import magic
import zipfile
import pdfplumber
import docx
import requests
from IPython.display import display
from google.colab import files

# OpenRouter API key here
API_KEY = "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"

# Upload file
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"Uploaded: {filename}")

# Detect FILE type
mime = magic.Magic(mime=True)
file_type = mime.from_file(filename)
print(f"Detected file type: {file_type}")

# -------- File Extractor Functions --------
def extract_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

def extract_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])

def extract_zip(file_path):
    text = ""
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall("temp_zip")
        for name in zip_ref.namelist():
            full_path = os.path.join("temp_zip", name)
            if os.path.isfile(full_path):
                text += extract_text_from_file(full_path)
    return text

def extract_strings_binary(file_path):
    with open(file_path, 'rb') as f:
        result = ""
        current = b""
        while byte := f.read(1):
            if 32 <= ord(byte) <= 126:
                current += byte
            else:
                if len(current) >= 4:
                    result += current.decode('utf-8', errors='ignore') + "\n"
                current = b""
    return result

def extract_text_from_file(file_path):
    mime_type = magic.Magic(mime=True).from_file(file_path)
    if mime_type == "application/pdf":
        return extract_pdf(file_path)
    elif mime_type in ["application/vnd.openxmlformats-officedocument.wordprocessingml.document"]:
        return extract_docx(file_path)
    elif mime_type == "application/zip":
        return extract_zip(file_path)
    elif "application/x-dosexec" in mime_type or "octet-stream" in mime_type:
        return extract_strings_binary(file_path)
    elif mime_type.startswith("text"):
        with open(file_path, 'r', errors='ignore') as f:
            return f.read()
    else:
        return f"[Binary or unknown file type: {mime_type}]"

# -------- LLM Scan Prompt --------
def scan_with_openrouter(model_id, text):
    prompt = f"""
You are a Antivirus software. Analyze the following file contents text and determine if there are any  malicious code, malware,miner,hacker code present in the textual content. only flag malicious when the condition is very  critical
also "This file may contain a mix of safe and unsafe code. Identify even small sections that may be insecure, even if most of the code appears normal."
Give answer in one work only: "Malicious" or "Safe",followed by a very very short explanation.

Content:
{text[:4000]}
"""

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "HTTP-Referer": "https://yourdomain.com",
        "X-Title": "Malware Detector Prototype",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code} - {response.text}"

# -------- Call Each Model Independently --------
try:
    content = extract_text_from_file(filename)
    print("Extracted content (truncated):")
    print(content[:500])

    # Define model names (OpenRouter IDs)
    models = {
        "Meta: Llama 3.3 8B Instruct  ": "meta-llama/llama-3.3-8b-instruct:free",
        "Meta: Llama 4 Scout": "meta-llama/llama-4-scout:free",
        "Meta: Llama 4 Maverick": "meta-llama/llama-4-maverick:free"
    }
#--------------- Giving the decision ----------
    print("\n=== Scan Results ===")
    for model_name, model_id in models.items():
        print(f"\n🔍 Model: {model_name}")
        verdict = scan_with_openrouter(model_id, content)
        print(verdict)

except Exception as e:
    print(f"Error: {e}")


Saving colab.txt to colab (26).txt
Uploaded: colab (26).txt
Detected file type: text/plain
Extracted content (truncated):
app.js
import React, { useState, useEffect } from "react";
import {
  ChakraProvider,
  Box,
  extendTheme,
  Button,
  Avatar,
  Flex,
  Text,
} from "@chakra-ui/react";
import { GoogleLogin } from "@react-oauth/google";
import { jwtDecode } from "jwt-decode";

import AIModelSelector from "./components/AIModelSelector";
import ChatHistory from "./components/ChatHistory";
import PromptBox from "./components/PromptBox";
import Advertisement from "./components/Advertisement";
import AIResponseBox 

=== Scan Results ===

🔍 Model: Meta: Llama 3.3 8B Instruct  
Safe, no suspicious code found but potential security risk due to use of localStorage and JWT decoding.

🔍 Model: Meta: Llama 4 Scout
Safe 

No critical malicious code, malware, miner, or hacker code is detected in the provided file content. The code appears to be a ReactJS application using Chakra UI components, 